**Laboratory Lecture 7**

## EXAMPLE: Causal effect of rain on wet grass

Let's consider again our Sprinkler network, assuming this is a reliable description of the causal relationships between its four variables:


<img src='https://drive.google.com/uc?id=1C2vyT2X19C_IekWf3rELuF4SumvJxo2b'>

In this case we want to estimate the causal effects of the rain on the "wetness" of the grass.

Note that it wouldn't be physically possible to modify the rain variable $R$. Yet, we can use probabilities from observational data of the weather to compute its causal effect "as if" we were able to intervene on it.

To this end, we can compute the effect of the intervention $P(G = true | do(R = true))$, or simply $P(g | do(r))$, by using the adjustment formula for the only parent of $R$, which is $C$:

$
\begin{align*}
P(g | do(r)) &= \sum_{z \in C} P(g | r, z) P(z) \\
&= P(g | r, c) P(c) + P(g | r, \neg c) P(\neg c)
\end{align*}
$

The probability distribution ${\bf P}(C) = \langle P(c), P(\neg c) \rangle$ is already given by the network. The conditional distribution ${\bf P}(G | r, c) = \langle P(g | r, c), P(\neg g | r, c)\rangle$ can be computed as follows:

$
\begin{align*}
{\bf P}(G | r, c) = \dfrac{{\bf P}(G, r, c)}{P(r, c)} = \alpha {\bf P}(G, r, c) &= \alpha \sum_s {\bf P}(G, r, c, s) \\
&= \alpha \sum_s P(c) P(r|c) P(s|c) {\bf P}(G | r, s) \\
&= \alpha P(c) P(r|c) \sum_s  P(s|c) {\bf P}(G | r, s) \\
&= \alpha' \sum_s  P(s|c) {\bf P}(G | r, s) \\
&= \alpha' \big[ P(s|c) {\bf P}(G | r, s) + P(\neg s|c) {\bf P}(G | r, \neg s) \big]
\end{align*}
$

where in the last steps we moved the product $P(c) P(r|c)$ within the normalisation factor $\alpha'$.

Substituting the values from the network's CPTs, we get the following:

$
{\bf P}(G | r, c) = \alpha' \big[ 0.1 \times \langle 0.95, 0.05 \rangle + 0.9 \times \langle 0.90, 0.10 \rangle \big] = \langle 0.905, 0.095 \rangle
$

If we do the same for the condition $\neg c$, we obtain the following distribution:

$
{\bf P}(G | r, \neg c) = \alpha' \big[ 0.5 \times \langle 0.95, 0.05 \rangle + 0.5 \times \langle 0.90, 0.10 \rangle \big] = \langle 0.925, 0.075 \rangle
$

Finally, we use the calculated values, $P(g | r, c) = 0.905$ and $P(g | r, \neg c) = 0.925$, in the previous adjustment formula and obtain the following:

$
P(g | do(r)) = 0.905 \times 0.5 + 0.925 \times 0.5 = 0.915
$

which is our causal effect of the intervention $R = true$ on the wetness $G = true$.




## RESOLUTION WITH PyAgrum

In [ ]:
!pip install pyAgrum
from IPython.display import display, Math, Latex

import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.1 MB/s eta 0:00:00


In [ ]:
# define the network
m1 = gum.fastBN("Cloudy{T|F}->Rain{T|F}->GrassWet{T|F}<-Sprinkler{T|F}<-Cloudy{T|F}")
m1.cpt("Cloudy")[:]=[0.5,0.5]
m1.cpt("Rain")[:]=[[0.8,0.2],  #Cloudy=T
                   [0.1,0.9]]  #Cloudy=F
m1.cpt("Sprinkler")[:]=[[0.1,0.9],  #Cloudy=T
                   [0.5,0.5]]  #Cloudy=F

m1.cpt("GrassWet")[{'Sprinkler':'T','Rain':'T'}]=[0.95,0.05]
m1.cpt("GrassWet")[{'Sprinkler':'T','Rain':'F'}]=[0.90,0.10]
m1.cpt("GrassWet")[{'Sprinkler':'F','Rain':'T'}]=[0.90,0.10]
m1.cpt("GrassWet")[{'Sprinkler':'F','Rain':'F'}]=[0.10,0.90]

gnb.flow.row(m1,m1.cpt("Cloudy"),m1.cpt("Rain"),m1.cpt("Sprinkler"), m1.cpt("GrassWet"))

In [ ]:
# let's compute P(g | do(r))
d1 = csl.CausalModel(m1)
cslnb.showCausalImpact(d1, "GrassWet", doing="Rain",values={"Rain" : "T"})
prob= cslnb.getCausalImpact(d1, "GrassWet", doing="Rain",values={"Rain" : "T"})
print(prob)

<IPython.core.display.HTML object>


In [ ]:
from IPython.core.display import HTML
import pandas as pd

# assume html_object is an HTML string or an IPython.core.display.HTML object
if isinstance(prob, HTML):
    html_object = prob.data

# read the HTML data into a Pandas dataframe
df_list = pd.read_html(html_object)

# assume the first dataframe in the list contains the data you're interested in
df = df_list[0]
print(df)

# Exract the target value
print(df.loc[0][0])


  GrassWet       
         T      F
0    0.915  0.085
0.915
